<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Decision-tree-classifier" data-toc-modified-id="Decision-tree-classifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Decision tree classifier</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#Testing-the-best-model" data-toc-modified-id="Testing-the-best-model-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Testing the best model</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [37]:
import pandas as pd
import numpy as np
import datetime
import time
from tqdm import notebook

import nltk
import torch
import transformers
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertConfig

from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
df = pd.read_csv('datasets/toxic_comments.csv')
df = df.sample(10000).reset_index()
display(df.head())
display(df.toxic.value_counts())

,index,text,toxic
0,67989,Article \n\nI can't remember my password as Ji...,0
1,145328,You've been quite about the merger for 5 days....,0
2,24311,""" (UTC)\n\nREVISED UPDATE: As far as I can tel...",0
3,127839,", 17 February 2011 (UTC)\n\nCarol and BigHex c...",0
4,27604,"""OMG. You read my mind. I was going to ask you...",0


0    9034
1     966
Name: toxic, dtype: int64

Оставили только 10000 текстов, иначе токенизация занимает слишком большое время.

In [7]:
%%time
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenized = df['text'].apply(lambda x: tokenizer.encode(x, truncation=True, max_length=500, add_special_tokens=True))
max_len = tokenized.apply(len).max()
padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Wall time: 20.8 s


In [9]:
model = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
#forming the embeddings
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [14]:
#splittimg dataset into train and test
features = np.concatenate(embeddings)
features_train, features_test, target_train, target_test = train_test_split(features, df['toxic'], test_size=.25)

## Обучение
### Logistic Regression

In [34]:
clf = LogisticRegression(random_state=25, max_iter=1000)
model = GridSearchCV(clf, param_grid={}, scoring='f1', n_jobs=7, cv=4)
model.fit(features_train, target_train)

print(f'F1-score: {model.best_score_:.3f}')
# pred = clf.predict(features_train)
# print("F1-score:", f1_score(target_train, pred))

C:\ProgramData\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


F1-score: 0.680


### Decision tree classifier

In [30]:
grid = {'max_depth': range(1,16,5), 'n_estimators':range(1,101,20)}
forest = RandomForestClassifier(random_state=25)
model = GridSearchCV(forest, param_grid=grid, scoring='f1', n_jobs=7, cv=4)
model.fit(features_train, target_train)

print(model.best_params_)
print(f'Best F1-score: {model.best_score_:.3f}')

{'max_depth': 11, 'n_estimators': 61}
Best F1-score: 0.504


### LightGBM

In [39]:
grid = {'max_depth': range(1,16,5), 'n_estimators':range(100,1001,100)}
lgbm = LGBMClassifier(random_state=25)
model = GridSearchCV(lgbm, param_grid=grid, scoring='f1', n_jobs=7, cv=4)
model.fit(features_train, target_train)

print(model.best_params_)
print(f'Best F1-score: {model.best_score_:.3f}')

{'max_depth': 1, 'n_estimators': 1000}
Best F1-score: 0.655


### Testing the best model

In [44]:
clf = LogisticRegression(random_state=25, max_iter=1000)
clf.fit(features_train, target_train)
pred = clf.predict(features_test)
print("F1-score:", round(f1_score(target_test, pred), 3))

F1-score: 0.7


## Выводы
Для анализа токсичности твитов выбрали отобрали 10 тыс. твитов из предложенного датасета. Далее с помощью модели BERT токенизировали тексты (использовали предобученную модель 'distilbert-base-uncased' из библиотеки pytorch_pretrained_bert).
На олученнном векторном представлении текстов тренировали модели и подбирали гиперпараметры для моделей логистической регрессии, случайного леса и lightGBM. Лучшее значение на тренировочных данных показала модель (значение метрики F1 0.68). На тестовых данных данная модель показывает значение метрики 0.70. 

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- []  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны